In [1]:
import pathlib
import urllib
import sys
import pandas as pd
import numpy as np
from scipy.stats import trim_mean, iqr, skew, kurtosis

# 1. Collect data

In [5]:
path = pathlib.Path('../data/adult.csv')
if path.exists():
    print('adult dataset found!')
else:
    sys.stdout.write('Downloading the adult dataset from the Internet...')
    ADULTURL = "https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data"
    urllib.request.urlretrieve(ADULTURL, path.absolute())
    sys.stdout.write('Done!')

adult dataset found!


# 2. Understand the data in context


In [12]:
# Load the adult dataset into a Pandas dataframe
adult_columns = ['age', 'workclass', 'fnlwgt', 'education', 'education-num', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'capital-gain', 'capital-loss', 'hours-per-week', 'native-country', '>50K']
adult = pd.read_csv(path.absolute(),names=adult_columns)

# 3. Understand the data
### 3.1 Describe the meansing and type for each attribute
 - **age**: numeric, continuous. 
 - **workclass**: categorical, discrete.
 - **fnlwgt**: numeric, continuous
 - **education**: categorical, discrete
 - **education-num**: numeric, continuous
 - **marital-status**: categorical, discrete
 - **occupation**: categorical, discrete
 - **relationship**: categorical, discrete
 - **race**: categorical, discrete
 - **sex**: categorical, binary
 - **capital-gain**: numeric, continuous
 - **capital-loss**: numeric, continuous
 - **hours-per-week**: numeric, continuous
 - **native-country**: categorical, discrete
 - **>50k**: categorical, binary

*3.1 subsection: explanation for non-self-explanatory attributes*
 - **fnlwgt**: represents final weight. according to [2], it's the number of units that this record could represent in the 
 target population. In [1], the author explained that the final weight is controlled by three factors: a single cell estimatation
 of the population 16+ for each state; controls for Hispanic Origin by age and sex; controls by race, age and sex.
 - **education_num**: represents the number of years of education in total.[2]
 - **relationship**: represents the individual's role in it's family.[2]
 - **capital_gain** and **capital_loss**: represents the income and loss from non-salary ways, e.g. investment.[2]
 
 
### 3.2 Verify data quality
#### 3.2.1 duplication

In [14]:
# Check duplication and drop duplication via pandas built in function
check_duplication = adult.duplicated()
duplicated = adult[check_duplication]
print("There are "+str(len(duplicated.index))+" duplicated data entries found.(One copy of the data entry will be kept in the dataset)")
adult = adult.drop_duplicates()
print("After drop duplicate, there are "+str(len(adult.index))+ " data entries remains.")

There are 0 duplicated data entries found.(One copy of the data entry will be kept in the dataset)
After drop duplicate, there are 32537 data entries remains.


#### 3.2.2 missing values

In [9]:
# drop the indexes for which column has value '?'
unknown_row =[]

for i in range(len(adult.index)):
    if ('?' in adult.iloc[i]['workclass']) or ('?' in adult.iloc[i]['occupation']) or ('?' in adult.iloc[i]['native-country']):
        unknown_row.append(i)
adult = adult.drop(index = unknown_row)
adult = adult.reset_index(drop=True)

32561
30162


In [10]:
# print(set(adult['education']))

{' Some-college', ' Bachelors', ' 1st-4th', ' 10th', ' 7th-8th', ' Prof-school', ' Assoc-voc', ' 11th', ' Assoc-acdm', ' Preschool', ' 9th', ' Masters', ' 5th-6th', ' 12th', ' HS-grad', ' Doctorate'}


In [300]:
#change >50k col to binary values
for i in range(len(adult.index)):
    if '>50K' in adult.iloc[i]['>50K']:
        adult.iloc[i, adult.columns.get_loc('>50K')] = 1
    else:
        adult.iloc[i,adult.columns.get_loc('>50K')] = 0

In [303]:
adult.head(5)

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,>50K
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,0
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,0
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,0
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,0
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,0


### 1. Statistics about Numeric values

In [304]:
# select all columns with numeric values
adult_numeric = adult.select_dtypes(include=np.number)

In [305]:
adult_numeric.head(5)

,age,fnlwgt,education-num,capital-gain,capital-loss,hours-per-week,>50K
0,39,77516,13,2174,0,40,0
1,50,83311,13,0,0,13,0
2,38,215646,9,0,0,40,0
3,53,234721,7,0,0,40,0
4,28,338409,13,0,0,40,0


In [306]:
cnames = ['mean', 'median', 'mode', 'trimmed mean (p=20%)', 'min', 'max', 'range', 'std']
adult_numeric.agg(lambda x: pd.Series([np.mean(x), np.median(x),x.mode()[0], trim_mean(x, 0.2),x.min(),x.max(),x.max()-x.min(),x.std()], index=cnames))

,age,fnlwgt,education-num,capital-gain,capital-loss,hours-per-week,>50K
mean,38.437902,1.897938e+05,10.121312,1092.007858,88.372489,40.931238,0.248922
median,37.000000,1.784250e+05,10.000000,0.000000,0.000000,40.000000,0.000000
mode,36.000000,2.034880e+05,9.000000,0.000000,0.000000,40.000000,0.000000
trimmed mean (p=20%),37.360206,1.775348e+05,10.024423,0.000000,0.000000,40.859653,0.081556
min,17.000000,1.376900e+04,1.000000,0.000000,0.000000,1.000000,0.000000
max,90.000000,1.484705e+06,16.000000,99999.000000,4356.000000,99.000000,1.000000
range,73.000000,1.470936e+06,15.000000,99999.000000,4356.000000,98.000000,1.000000
std,13.134665,1.056530e+05,2.549995,7406.346497,404.298370,11.979984,0.432396


### 2. Statistics about Categorical values

In [307]:
# select all columns with categorical values
adult_categorical = adult.select_dtypes(include=['object'])

In [308]:
adult_categorical.head(5)

,workclass,education,marital-status,occupation,relationship,race,sex,native-country
0,State-gov,Bachelors,Never-married,Adm-clerical,Not-in-family,White,Male,United-States
1,Self-emp-not-inc,Bachelors,Married-civ-spouse,Exec-managerial,Husband,White,Male,United-States
2,Private,HS-grad,Divorced,Handlers-cleaners,Not-in-family,White,Male,United-States
3,Private,11th,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,United-States
4,Private,Bachelors,Married-civ-spouse,Prof-specialty,Wife,Black,Female,Cuba


In [309]:
cnames = adult_categorical.columns.tolist()
cnames

['workclass',
 'education',
 'marital-status',
 'occupation',
 'relationship',
 'race',
 'sex',
 'native-country']

In [310]:
adult_categorical['workclass'].value_counts()

 Private             22286
 Self-emp-not-inc     2499
 Local-gov            2067
 State-gov            1279
 Self-emp-inc         1074
 Federal-gov           943
 Without-pay            14
Name: workclass, dtype: int64

In [311]:
adult_categorical['education'].value_counts()

 HS-grad         9840
 Some-college    6678
 Bachelors       5044
 Masters         1627
 Assoc-voc       1307
 11th            1048
 Assoc-acdm      1008
 10th             820
 7th-8th          557
 Prof-school      542
 9th              455
 12th             377
 Doctorate        375
 5th-6th          288
 1st-4th          151
 Preschool         45
Name: education, dtype: int64

In [312]:
adult['marital-status'].value_counts()

 Married-civ-spouse       14065
 Never-married             9726
 Divorced                  4214
 Separated                  939
 Widowed                    827
 Married-spouse-absent      370
 Married-AF-spouse           21
Name: marital-status, dtype: int64

In [313]:
adult_categorical['occupation'].value_counts()

 Prof-specialty       4038
 Craft-repair         4030
 Exec-managerial      3992
 Adm-clerical         3721
 Sales                3584
 Other-service        3212
 Machine-op-inspct    1966
 Transport-moving     1572
 Handlers-cleaners    1350
 Farming-fishing       989
 Tech-support          912
 Protective-serv       644
 Priv-house-serv       143
 Armed-Forces            9
Name: occupation, dtype: int64

In [314]:
adult['relationship'].value_counts()

 Husband           12463
 Not-in-family      7726
 Own-child          4466
 Unmarried          3212
 Wife               1406
 Other-relative      889
Name: relationship, dtype: int64

In [315]:
adult['race'].value_counts()

 White                 25933
 Black                  2817
 Asian-Pac-Islander      895
 Amer-Indian-Eskimo      286
 Other                   231
Name: race, dtype: int64

In [316]:
adult['sex'].value_counts()

 Male      20380
 Female     9782
Name: sex, dtype: int64

In [317]:
adult['native-country'].value_counts()

 United-States                 27504
 Mexico                          610
 Philippines                     188
 Germany                         128
 Puerto-Rico                     109
 Canada                          107
 El-Salvador                     100
 India                           100
 Cuba                             92
 England                          86
 Jamaica                          80
 South                            71
 China                            68
 Italy                            68
 Dominican-Republic               67
 Vietnam                          64
 Guatemala                        63
 Japan                            59
 Poland                           56
 Columbia                         56
 Haiti                            42
 Taiwan                           42
 Iran                             42
 Portugal                         34
 Nicaragua                        33
 Peru                             30
 Greece                           29
 

In [319]:
adult.groupby('workclass').agg(['mean', lambda x: trim_mean(x, 0.2), 'median', 'std']).T

workclass                    Federal-gov      Local-gov        Private  \
age            mean            42.577943      41.743106      36.794355   
               <lambda_0>      42.433862      41.235294      35.464403   
               median          43.000000      41.000000      35.000000   
               std             11.532840      12.267402      12.842129   
fnlwgt         mean        185717.107105  188422.886309  192719.985776   
               <lambda_0>  171778.003527  178795.264303  180036.951914   
               median      176904.000000  178383.000000  181059.500000   
               std         117766.713221  100654.252367  105981.547011   
education-num  mean            10.948038      11.036768       9.871085   
               <lambda_0>      10.749559      11.099114       9.768920   
               median          10.000000      10.000000      10.000000   
               std              2.100565       2.545539       2.484040   
capital-gain   mean           832.321315     829.230285     879.858207   
               <lambda_0>       0.000000       0.000000       0.000000   
               median           0.000000       0.000000       0.000000   
               std           4111.473884    5375.040712    6338.117016   
capital-loss   mean           111.731707     110.323174      79.629498   
               <lambda_0>       0.000000       0.000000       0.000000   
               median           0.000000       0.000000       0.000000   
               std            451.148107     440.372500     383.173354   
hours-per-week mean            41.279958      41.002419      40.250875   
               <lambda_0>      40.014109      40.476229      40.488035   
               median          40.000000      40.000000      40.000000   
               std              8.720239      10.760325      11.264958   
>50K           mean             0.387063       0.294630       0.218792   
               <lambda_0>       0.312169       0.157937       0.031334   
               median           0.000000       0.000000       0.000000   
               std              0.487337       0.455987       0.413437   

workclass                   Self-emp-inc   Self-emp-not-inc      State-gov  \
age            mean            46.027933          45.011605      39.362002   
               <lambda_0>      45.595975          44.182545      38.738622   
               median          46.000000          44.000000      39.000000   
               std             12.689685          13.364069      12.386871   
fnlwgt         mean        175775.219739      175222.506603  184440.424550   
               <lambda_0>  163314.314241      165205.450366  170642.979194   
               median      164614.000000      168098.000000  170091.000000   
               std          97355.381441      100657.849119  111202.987584   
education-num  mean            11.167598          10.211285      11.349492   
               <lambda_0>      11.071207          10.094604      11.240572   
               median          10.000000          10.000000      10.000000   
               std              2.544997           2.756639       2.530994   
capital-gain   mean          4810.746741        1913.134454     684.306489   
               <lambda_0>       0.000000           0.000000       0.000000   
               median           0.000000           0.000000       0.000000   
               std          17828.904145       11074.584412    3771.592651   
capital-loss   mean           153.504655         115.572229      81.147772   
               <lambda_0>       0.000000           0.000000       0.000000   
               median           0.000000           0.000000       0.000000   
               std            546.910433         465.740922     388.739091   
hours-per-week mean            48.802607          44.432173      39.053948   
               <lambda_0>      47.907121          44.179214      39.728218   
               median          50.000000          40.000000

In [320]:
adult.cov()

,age,fnlwgt,education-num,capital-gain,capital-loss,hours-per-week,>50K
age,172.519419,-1.061753e+05,1.457834,7.797393e+03,319.498166,15.986877,1.374398
fnlwgt,-106175.339198,1.116255e+10,-12121.429540,3.298775e+05,-416454.241511,-28966.974882,-409.210210
education-num,1.457834,-1.212143e+04,6.502474,2.349739e+03,82.112167,4.659381,0.369689
capital-gain,7797.393396,3.298775e+05,2349.738644,5.485397e+07,-96506.651513,7136.545913,708.375220
capital-loss,319.498166,-4.164542e+05,82.112167,-9.650665e+04,163457.172378,253.881355,26.231868
hours-per-week,15.986877,-2.896697e+04,4.659381,7.136546e+03,253.881355,143.520022,1.188729
>50K,1.374398,-4.092102e+02,0.369689,7.083752e+02,26.231868,1.188729,0.186966


# Reference
[1] Kaggle adult census income dataset. Last access: Sept. 2019. url: https://www.kaggle.com/uciml/adult-census-income
[2] [2] Haojun Zhu, Predicting Earning Potential using the Adult Dataset. Dec. 2016. url: https://rstudio-pubs-static.s3.amazonaws.com/235617_51e06fa6c43b47d1b6daca2523b2f9e4.html